## LAB 1 ##  

This lab will see you connect from pyspark to Polaris catalog, creating a namespace and a table managed by Polaris

In the next cell you need to replace <user_env> with the directory of your anaconda environment. This allows the Jupyter notebook to find your spark environment.
It should look something like: `'/Users/mstarc/anaconda3/envs/iceberg-lab-techup/lib/python3.12/site-packages/pyspark'` when done.

You will also need to replace `credential` below. It will use the Polaris connection details you previously created in the form `<CLIENT ID>:<SECRET ID>`. It should look like: `EobogpKvVdz3STi1XP/R15oeyM8=:YSP51zJhSIFelzPyat5GGP74mVBdEfht7e71Cc2wlfk=`

In [3]:
import os
os.environ['SPARK_HOME'] = '<user_env>/lib/python3.12/site-packages/pyspark'

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('iceberg_lab') \
.config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.1,software.amazon.awssdk:bundle:2.20.160,software.amazon.awssdk:url-connection-client:2.20.160') \
.config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
.config('spark.sql.defaultCatalog', 'polaris') \
.config('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog') \
.config('spark.sql.catalog.polaris.type', 'rest') \
.config('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation','vended-credentials') \
.config('spark.sql.catalog.polaris.uri','https://tzb93977.snowflakecomputing.com/polaris/api/catalog') \
.config('spark.sql.catalog.polaris.credential','<CLIENT ID>:<SECRET>') \
.config('spark.sql.catalog.polaris.warehouse','apj_ps_tmup_int') \
.config('spark.sql.catalog.polaris.scope','PRINCIPAL_ROLE:admin_int') \
.config('spark.sql.catalog.polaris.client.region','us-west-2') \
.getOrCreate()

#Show namespaces
spark.sql("show namespaces").show()


24/10/01 22:09:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+
|namespace|
+---------+
|   mstarc|
+---------+



In the below cells replace `<username>` with your CAS2 login name or first part of your snowflake email (remove full stop)

In [ ]:

#Create namespace
spark.sql("create namespace <username>")



In [ ]:
#Use namespace
spark.sql("use namespace <username>")

#Show tables; this will show no tables since it is a new namespace
spark.sql("show tables").show()



In [ ]:
#create a test table
spark.sql("create table apj_ps_managed (col1 int) using iceberg");



In [ ]:
#insert a record in the table
spark.sql("insert into apj_ps_managed values (1)");

In [ ]:


#query the table
spark.sql("select * from apj_ps_managed").show();

Once you can see the data in your created table. Go back to Polaris and refresh the catalogs. You should see your namespace and newly created table. You can then move onto LAB2